## Running the 2x2 Simulation chain on Polaris using Singularity container

In [ ]:
#Obtain a debug node to do our work
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A ALCF_for_DUNE

In [ ]:
# Module load command for software loading
module use /soft/spack/gcc/0.6.1/install/modulefiles/Core
module load apptainer

#load python
module use /soft/modulefiles ; module load conda; conda activate base

In [ ]:
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER_DIR='/grand/ALCF_for_DUNE/2x2_sim_data/containers'
export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE

#change the directory to where you want the output files
export ARCUBE_OUTDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/out'
export ARCUBE_LOGDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/log'

In [ ]:
#Change this to your directory for up systematics
export ARCUBE_DIR='/home/aleena/2x2_sim_111424/2x2_sim'
#OR for down systematics
export ARCUBE_DIR='/home/aleena/2x2_productions_down/2x2_sim_01312025'

In [ ]:
#only once
git clone -b MiniRun6 https://github.com/DUNE/2x2_sim
cd 2x2_sim

'''
Make these changes in the files once you have it cloned (line with - is to be changed to the line with +)

1. run-validation/run_validation.sh
if [[ "$ARCUBE_RUNTIME" == "NONE" ]]; then
-    module load python/3.11
+    #module load python/3.11

2. run-validation/install_validation.sh
-module unload python 2>/dev/null
-module load python/3.11
+#module unload python 2>/dev/null
+#module load python/3.11

3. run-ndlar-flow/install_ndlar_flow.sh

-module load python/3.11
+# module load python/3.11

Aleena: You should keep the venv so ignore the below lines on commenting out the venv
-python -m venv "$venvName"
-source "$venvName"/bin/activate
+#python -m venv "$venvName"
+#source "$venvName"/bin/activate

4. run-larnd-sim/install_larnd_sim.sh
-module load cudatoolkit/11.7
-module load python/3.11
+# module load cudatoolkit/11.7
+#module load python/3.11
'''


#### IGNORE FOR MiniRun6 and beyond ###
NDLAR flow installation: Only because of the bug with pylandau, can skip in later MiniRun versions

Change pylandau package as it has some bugs (Solution provided by Matt Kramer)

- cd run-ndlar-flow
- Before installing the ndlar flow stage, comment out pip install -e . for ndlar_flow at the end of the install_ndlar_flow.sh
- Run the install script
- cd ndlar_flow directory inside run-ndlar-flow
- source ../flow.venv/bin/activate
- Replace `pylandau` with `pylandau @ git+https://github.com/cuddandr/pylandau.git#egg=pylandau` in ndlar_flow/setup.py 
- replace `pylandau` with `git+https://github.com/cuddandr/pylandau.git` in ndlar_flow/env-nompi.yaml and ndlar_flow/env.yaml (optional ndlar_flow/test/env-nompi.yaml and ndlar_flow/test/env.yaml)
- Make sure you are inside run-ndlar-flow/ndlar_flow, pip install -e .
- exit out of venv by `deactivate` command

In [ ]:
#run this command to install tmp_bin
mkdir -p tmp_bin
cp /usr/bin/time tmp_bin/

#if not installed already isntall ndlarflow
cd run-ndlar-flow
./install_ndlar_flow.sh
cd ..
#if not installed already isntall ndlarflow as above

#install larnd-sim
cd run-larnd-sim
./install_larnd_sim.sh
cd ..

#install convert2h5
export ARCUBE_CONTAINER='genie_edep.3_04_00.20230912.sif'
cd run-convert2h5
./install_convert2h5.sh
cd ..

#install validation
cd run-validation
./install_validation.sh
cd ..


#### IGNORE FOR MiniRun6 and beyond ###
The below command is to run singularity container manually to execute our bash scripts. Ideally the run scripts (run_genie, run_edep_sim, etc) should take care of loading the container and then running the script but right now for some reason it keeps calling the singularity command in a loop (looking into this)

> [!IMPORTANT]  
> once the above problem is resolved, we dont need to wouldn't need to run the below command manually

In [ ]:
'''
#### IGNORE FOR MiniRun6 and beyond ###
#once the above problem is resolved, we dont need to wouldn't need to run the below command manually


##THE ABOVE PROBLEM IS SOLVED, DO NOT NEED THESE COMMANDS
export ARCUBE_CONTAINER='genie_edep.3_04_00.20230912.sif'
singularity exec -B $ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE $ARCUBE_CONTAINER_DIR/$ARCUBE_CONTAINER /bin/bash
export ARCUBE_CONTAINER='NONE'
export ARCUBE_RUNTIME='NONE'

'''
#IGNORE ABOVE COMMANDS, MAKE SURE POINT 5 OF CELL 4 IS MADE

In [ ]:
'''
GENIE Stage-nu
'''

cd run-genie
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='genie_edep.3_04_00.20230912.sif'
export ARCUBE_DET_LOCATION='MiniRun5-Nu'
export ARCUBE_OUT_NAME='genie.nu'
export ARCUBE_DK2NU_DIR='/grand/ALCF_for_DUNE/users/fathima/2x2_sim_data/NuMI_dk2nu/newtarget-200kA_20220409'
export ARCUBE_EXPOSURE='1E15'
export ARCUBE_GEOM='geometry/Merged2x2MINERvA_v4/Merged2x2MINERvA_v4_noRock.gdml'
export ARCUBE_TUNE='AR23_20i_00_000'
export ARCUBE_XSEC_FILE='/grand/ALCF_for_DUNE/users/fathima/2x2_sim_data/gxspl-NUsmall.xml'
export ARCUBE_RUN_OFFSET=0
export ARCUBE_INDEX=0


./run_genie.sh

In [ ]:
'''
GENIE Stage-rock
'''

cd run-genie
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='genie_edep.3_04_00.20230912.sif'
export ARCUBE_OUTDIR_BASE='/grand/ALCF_for_DUNE/users/fathima/out.test'
export ARCUBE_LOGDIR_BASE='/grand/ALCF_for_DUNE/users/fathima/logs_cmd.test'
export ARCUBE_DET_LOCATION='MiniRun5-Rock'
export ARCUBE_OUT_NAME='genie.rock'
export ARCUBE_DK2NU_DIR='/grand/ALCF_for_DUNE/users/fathima/2x2_sim_data/NuMI_dk2nu/newtarget-200kA_20220409'
export ARCUBE_EXPOSURE='1E15'
export ARCUBE_GEOM='geometry/Merged2x2MINERvA_v4/Merged2x2MINERvA_v4_justRock.gdml'
export ARCUBE_TUNE='AR23_20i_00_000'
export ARCUBE_XSEC_FILE='/grand/ALCF_for_DUNE/users/fathima/2x2_sim_data/gxspl-NUsmall.xml'
export ARCUBE_RUN_OFFSET=1000000000
export ARCUBE_INDEX=0

./run_genie.sh


In [ ]:
#exit out of above container


##THE PROBLEM IS SOLVED, DO NOT NEED THESE COMMANDS

#load the below container for the next few steps
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
singularity exec -B $ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE $ARCUBE_CONTAINER_DIR/$ARCUBE_CONTAINER /bin/bash
export ARCUBE_CONTAINER='NONE'
export ARCUBE_RUNTIME='NONE'

#IGNORE ABOVE COMMANDS, MAKE SURE POINT 5 OF CELL 4 IS MADE

In [ ]:
'''
Edepsim Stage-nu
'''

cd run-edep-sim
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_GENIE_NAME='genie.nu'
export ARCUBE_EDEP_MAC='macros/2x2_beam.mac'
export ARCUBE_GEOM_EDEP='geometry/Merged2x2MINERvA_v4/Merged2x2MINERvA_v4_withRock.gdml'
export ARCUBE_RUN_OFFSET=0
export ARCUBE_OUT_NAME='edep.nu'
export ARCUBE_INDEX=0


./run_edep_sim.sh

In [ ]:
'''
Edepsim Stage-rock
'''

cd run-edep-sim
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_GENIE_NAME='genie.rock'
export ARCUBE_EDEP_MAC='macros/2x2_beam.mac'
export ARCUBE_GEOM_EDEP='geometry/Merged2x2MINERvA_v4/Merged2x2MINERvA_v4_withRock.gdml'
export ARCUBE_RUN_OFFSET=1000000000
export ARCUBE_OUT_NAME='edep.rock'
export ARCUBE_INDEX=0

./run_edep_sim.sh

In [ ]:
'''
Hadd Stage-nu
'''

cd run-hadd
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_IN_NAME='edep.nu'
#only one file produced, increase this to 10 to hadd together 10 1e15 files to form a 1e16 root file
export ARCUBE_HADD_FACTOR=1
export ARCUBE_OUT_NAME='edep.nu.hadd'
export ARCUBE_INDEX=0

./run_hadd.sh

In [ ]:
'''
Hadd Stage-rock
'''

cd run-hadd
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_IN_NAME='edep.rock'
#only one file produced, increase this to 10 to hadd together 10 1e15 files to form a 1e16 root file
export ARCUBE_HADD_FACTOR=1
export ARCUBE_OUT_NAME='edep.rock.hadd'
export ARCUBE_INDEX=0

./run_hadd.sh

In [ ]:
'''
spill build
'''

cd run-spill-build
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_NU_NAME='edep.nu.hadd'
#change pot to 1e16 if your hadd-ed files have 1e16 POT, in this tutorial only 1e15
export ARCUBE_NU_POT=1E15
export ARCUBE_ROCK_NAME='edep.rock.hadd'
export ARCUBE_ROCK_POT=1E15
export ARCUBE_OUT_NAME='Tutorial.spill'
export ARCUBE_INDEX=0

./run_spill_build.sh

In [ ]:
'''
convert2h5
'''

cd run-convert2h5
export ARCUBE_RUNTIME='SINGULARITY'
# export ARCUBE_CONTAINER='genie_edep.3_04_00.20230912.sif'
export ARCUBE_CONTAINER='sim2x2_ndlar011.sif'
export ARCUBE_ACTIVE_VOLUME='volTPCActive'
export ARCUBE_SPILL_NAME='spill'
export ARCUBE_OUT_NAME='convert2h5'
export ARCUBE_INDEX=0

./run_convert2h5.sh

Change the location of LIGHT_LUT FILE in `run-larnd-sim/larnd-sim/larndsim/config/config.yaml` in 4 places  
`/global/cfs/cdirs/dune/www/data/2x2/simulation/larndsim_data/light_LUT/lightLUT_Mod0.npz` -> `/grand/ALCF_for_DUNE/users/aleena/products/lightLUT_Mod0.npz`   
`/global/cfs/cdirs/dune/www/data/2x2/simulation/larndsim_data/light_LUT/lightLUT_Mod123.npz` -> `/grand/ALCF_for_DUNE/users/aleena/products/lightLUT_Mod123.npz`

In [ ]:
'''
larnd
'''

#No container required for this stage onwards


#------------------------------------------------------------------------------------------------------
#Only if you havent gotten a node yet to workon, you will need to ask for compute nodes to work on GPU 
# and reload enviornment variables    

#Obtain a debug node to do our work
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A ALCF_for_DUNE

export ARCUBE_CONTAINER_DIR='/grand/ALCF_for_DUNE/2x2_sim_data/containers'
#change the directory to where you want the output files
export ARCUBE_OUTDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/out'
export ARCUBE_LOGDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/log'

#------------------------------------------------------------------------------------------------------
#Change this to your directory for up systematics
export ARCUBE_DIR='/home/aleena/2x2_sim_111424/2x2_sim'
#OR for down systematics
export ARCUBE_DIR='/home/aleena/2x2_productions_down/2x2_sim_01312025'
#------------------------------------------------------------------------------------------------------

#load conda and cuda
module use /soft/modulefiles ; module load conda; conda activate base
module load cudatoolkit-standalone/11.8.0

cd run-larnd-sim

export ARCUBE_RUNTIME='NONE'
export ARCUBE_CONVERT2H5_NAME='convert2h5'
export ARCUBE_OUT_NAME='larnd'
export ARCUBE_LARNDSIM_CONFIG='2x2_old_response'
export ARCUBE_INDEX=0

./run_larnd_sim.sh

In [ ]:
'''
NDLARFLOW
'''

#load conda and cuda
module use /soft/modulefiles ; module load conda; conda activate base
cd run-ndlar-flow


export ARCUBE_RUNTIME='NONE'
export ARCUBE_IN_NAME='larnd'
export ARCUBE_OUT_NAME='flow'
export ARCUBE_INDEX=0

./run_ndlar_flow.sh

In [ ]:
'''
validation
'''

#load conda for python
module use /soft/modulefiles ; module load conda; conda activate base

#if validation.venv is not present due to failure of admin/install_everyhting, run install script manually

#load conda and cuda
cd run-validation

export ARCUBE_RUNTIME='NONE'
export ARCUBE_EDEP_NAME='convert2h5'
export ARCUBE_LARND_NAME='larnd'
export ARCUBE_FLOW_NAME='flow'
export ARCUBE_OUT_NAME='plots'
export ARCUBE_INDEX=0

./run_validation.sh